<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Macro_Prediction_Models/addingHourlyTraffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tqdm
import datetime
import pickle
from ast import literal_eval
import numpy as np
import calendar
import os

In [ ]:
#initialization
Vehicle_Type = 'Electric_Vehicles'
Vehicle_ID = [751]

dateFormat = '%Y-%m-%d'
datetimeFormat = '%Y-%m-%d %H:%M:%S:%f'

Loading data for mapping OSM segments to TMC IDs

In [ ]:
# load the OSM_TMC_MAP
OSM_TMC_MAP_PATH = os.path.join(os.getcwd(), "data", "osm_tmc_matching_ids.pickle")
with open(OSM_TMC_MAP_PATH, 'rb') as handle:
    OSM_TMC_MAP = pickle.load(handle)



Loading Hourly Traffic Data for Chattanooga

In [ ]:
df_TMC = pd.read_csv(f'Chattanooga_TrafficData_August19_July20.csv')
print(df_TMC.columns)

df_TMC=df_TMC.dropna()

Columns = ['Speed_Real','Speed_FreeFlow','Speed_JF','Hour']

for col in df_TMC.columns:
    if col in Columns:
        df_TMC[col] = df_TMC[col].apply(literal_eval)


TMC_Id_for_Matching = list(df_TMC.TMC)
Day = list(df_TMC.Day)
Hour = list(df_TMC.Hour)
Date = list(df_TMC.Date)

Hourly_Speed_Real = list(df_TMC.Speed_Real)
Hourly_Speed_Freeflow = list(df_TMC.Speed_FreeFlow)
Hourly_Jam_Factor = list(df_TMC.Speed_JF)

In [ ]:
def findDay(year, month, day):
        dayNumber = calendar.weekday(year, month, day)
        days = ["Monday", "Tuesday", "Wednesday", "Thursday",
                "Friday", "Saturday", "Sunday"]
        return (dayNumber)

Mapping OSM to TMC ID

In [ ]:
Vehicle_Name = f'BYD_751'
print(f'Processing {Vehicle_Name}')

df = pd.read_csv(f'{Vehicle_Name}_with_Elevation_Weather.csv', low_memory=False)
print(len(df))

OSM_Feature = list(df.OSM_Feature)

TMC_Id = []
OSM = []
Found_OSM = []
Not_Found = 0
for i in OSM_Feature:
    i = str(i)
    temp = []
    for key, value in OSM_TMC_MAP.items():
        if i == key:
            temp.append(value)
    if len(temp) != 0:
        for j in temp:
            Found_OSM.append(i)
            TMC_Id.append(j)
    else:
        Not_Found += 1
        OSM.append(i)
        TMC_Id.append(0)


print(f'Total Segments = {len(TMC_Id)}\n TMC-ID not found = {Not_Found}')
print(f'Total Unique OSM =  {len(set(OSM_Feature))}\n Mapped to TMC = {len(set(Found_OSM))} \n')

TMC_Id = np.array(TMC_Id)
df['TMC_Id'] = TMC_Id


Breaking up TimeStamps into Time of Day and Day of Week

In [ ]:
Time = df.TimeStart
Lat_ViriCity = df.Initial_recorded_Latitude
Long_ViriCity = df.Initial_recorded_Longitude
Date_from_ViriCity = []
Hour_from_ViriCity = []

Day_of_Week = []
Time_of_Day = []
for i in Time:
    timestamp = i
    TD = datetime.datetime.strptime(timestamp, datetimeFormat)
    Date_from_ViriCity.append(TD.date())

    year = TD.year
    month = TD.month
    day = TD.day

    Day_of_Week.append(findDay(year, month, day))

    Time_of_Day.append(TD.hour)


df['Day_of_Week'] = Day_of_Week
df['Time_of_Day'] = Time_of_Day

Matching with TMC and adding Hourly Traffic Data

In [ ]:
TMC_Id = list(df.TMC_Id)
Day_of_Week = list(df.Day_of_Week)
Time_of_Day = list(df.Time_of_Day)
TimeStart = list(df.TimeStart)


Speed_Ratio = []
Jam_Factor = []

for count in tqdm.tqdm(range(len(TMC_Id))):
    tmc = TMC_Id[count]
    indi_tmc = []
    timestamp = TimeStart[count]

    TD = datetime.datetime.strptime(timestamp, datetimeFormat)
    date_viricity = TD.date()
    hour_viricity = TD.hour

    if len(tmc) > 2:
        tmc = tmc.replace("', '", ",")
        tmc = tmc.replace("'", "")
        tmc = tmc.replace("[", "")
        tmc = tmc.replace("]", "")
        tmc = tmc.split(',')
        for j in tmc:
            indi_tmc.append(j)
        if len(indi_tmc)>0:
            for indi in indi_tmc:
                tag = 0
                tmp_JF = []
                tmp_SR = []
                for i in range(len(TMC_Id_for_Matching)):
                    date_time_str = Date[i]

                    TD = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')
                    Date_Traffic = TD.date()
                    hour_Traffic = TD.hour
                    if indi==TMC_Id_for_Matching[i] and date_viricity==Date_Traffic:

                        hour_list=Hour[i]
                        JF = Hourly_Jam_Factor[i]
                        RS = Hourly_Speed_Real[i]
                        FF = Hourly_Speed_Freeflow[i]

                        for h in hour_list:
                            if h==hour_viricity:
                                index = hour_list.index(h)
                                tmp_JF.append(JF[index])
                                ratio = RS[index]/FF[index]
                                tmp_SR.append(ratio)
                                tag = 1
                if tag != 1:
                    tmp_SR.append(1)
                    tmp_JF.append(0)

            Jam_Factor.append(sum(tmp_JF)/len(tmp_JF))
            Speed_Ratio.append(sum(tmp_SR)/len(tmp_SR))
        else:
            Speed_Ratio.append(1)
            Jam_Factor.append(0)
    else:
        Speed_Ratio.append(1)
        Jam_Factor.append(0)



df['Speed_Ratio'] = Speed_Ratio
df['Jam_Factor'] = Jam_Factor

df.to_csv(f'{Vehicle_Name}_with_Elevation_Weather_Traffic_Day_Week.csv',index=False)